In [ ]:
using Symbolics

: 

In [2]:
function integrateTaylorPolynomials(eq::Num, x::Num)
    # this function works only for (positive) polynomials 
    #
    # the function needs only Symbolics.jl
    #
    # Since the Symbolics.coeff does not support expression with division, lowestNegativeOrder::Int option should be implemented afterwards

    lowestNegativeOrder::Int=0
    eq = simplify(eq;expand=true,polynorm=true)
    eqval=Symbolics.value(eq)
    

    # The above statement is to switch off the negative orders integral
    
    if Symbolics.isdiv(eqval)
        eq=eqval.num 
        denominator=eqval.den 
    else 
        denominator =1
    end
    highestOrder=Symbolics.degree(eq,x)
   
    tmpeq = eq
    neweq = 0
    if highestOrder>0
        for i in 1:highestOrder
            old_coef=Symbolics.coeff(eq,x^i)
            tmpeq-=old_coef*x^i
            new_coef=old_coef//(i+1)
            neweq+=new_coef*x^(i+1)        
        end
    end
    
    if lowestNegativeOrder<0
        old_coef=Symbolics.coeff(eq,x^(-1))
        tmpeq-=old_coef*x^(-1)
        new_coef=old_coef
        neweq+=new_coef*log(x)

        if lowestNegativeOrder<1
            for i in lowestNegativeOrder:-2
                old_coef=Symbolics.coeff(eq,x^i)
                tmpeq-=old_coef*x^i
                new_coef=old_coef//(i+1)
                neweq+=new_coef*x^(i+1)
            end
        end
    end

    # the rest of tmpeq should be the constant
    neweq+=tmpeq*x
    neweq/=denominator
    return simplify(neweq;expand=true)
end

integrateTaylorPolynomials (generic function with 1 method)

In [3]:
function ignoreHigherOrderTerms(eq::Num,x::Num,highestOrderForConsideration::Int)
    eq = simplify(eq;expand=true,polynorm=true)
    eqval=Symbolics.value(eq)


    # The above statement is to switch off the negative orders integral
    
    if Symbolics.isdiv(eqval)
        eq=eqval.num 
        denominator=eqval.den 
    else 
        denominator =1
    end
    highestOrder=Symbolics.degree(eq,x)

    highestOrderForConsideration > 0 || error("highestOrderForConsideration should be positive")
    highestOrderForConsideration<highestOrder || return eq/denominator
    tmpeq=eq
    for i in highestOrderForConsideration+1:highestOrder

        old_coef=Symbolics.coeff(eq,x^i)

        tmpeq-=old_coef*x^i
    end
    neweq=tmpeq/denominator
    return simplify(neweq; expand=true)
end
        

ignoreHigherOrderTerms (generic function with 1 method)

In [4]:
orderUtime=4
orderUspace=4
#m = 15 # the midpoint to evaluate the operator at
L₁ₗ = 1 # the leftmost point for x1 direction
L₁ᵣ = 1 # the rightmost point
L₂ₗ = 1 # the leftmost point for x2 direction
L₂ᵣ = 1 # the right most point
Lₜₗ = 1 # the leftmost point for t direction
Lₜᵣ = 1 # the rightmost point


orderStructure=L₁ₗ+L₁ᵣ # number of nodes minus one

@variables dx₁ dx₂ dt   
@variables Cᴾ[1:orderStructure+1,1:orderStructure+1] Cᴹ[1:2,1:2,1:orderStructure+1,1:orderStructure+1] Cᵘ[1:orderUspace+1,1:orderStructure+1,1:orderUtime+1] # These are the derivatives at the node point xₘ, tₙ
#@variables x¹ t




#@variables ξ¹[1:L₁ₗ+L₁ᵣ+1]  τ[1:Lₜₗ+Lₜᵣ+1] # These are the node points in the spatial and time domain
#@variables ξ¹ₘ[-L₁ₗ:L₁ᵣ] τₙ[-L₁ₗ:L₁ᵣ] ρₘ[-L₁ₗ:L₁ᵣ] μₘ[-L₁ₗ:L₁ᵣ]
ξ¹ = Symbolics.variables(:ξ¹,-L₁ₗ:L₁ᵣ) # nodes in space (x₁ direction)
ξ² = Symbolics.variables(:ξ²,-L₂ₗ:L₂ᵣ) # nodes in space (x₁ direction)
τ = Symbolics.variables(:τ,-Lₜₗ:Lₜᵣ) # nodes in time
ρ = Symbolics.variables(:ρ,-L₁ₗ:L₁ᵣ)
Λ = Symbolics.variables(:Λ,1:2,1:2,-L₁ₗ:L₁ᵣ,-L₂ₗ:L₂ᵣ)
u= Symbolics.variables(:u,1:2,-L₁ₗ:L₁ᵣ,-L₂ₗ:L₂ᵣ,-Lₜₗ:Lₜᵣ)
A= Symbolics.variables(:A,-L₁ₗ:L₁ᵣ,-L₂ₗ:L₂ᵣ,-Lₜₗ:Lₜᵣ)


ρ(dx₁,dx₂) = sum(sum(Cᴾ[i] * dx₁^(i-1) * dx₂^(j-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
μ(dx₁,dx₂) = sum(Cᴹ[i] * dx₁^(i-1) / factorial(BigInt(i-1)) for i in 1:orderStructure+1)
u(dx₁,dt) = sum(sum(Cᵘ[i,j] * dx₁^(i-1) * dt^(j-1) / factorial(BigInt(i-1)) / factorial(BigInt(j-1)) for i in 1:orderUspace+1) for j in 1:orderUtime+1)

u (generic function with 1 method)